In [1]:
import os
from Bio import SeqIO

def read_data():
    base_directory = './e_coli_paidb/'
    data = {}
    
    for name in os.listdir(base_directory):
        dir_path = os.path.join(base_directory, name)

        if os.path.isdir(dir_path):
            fasta_file = os.path.join(dir_path, f"{name}.fasta")
            pai_file   = os.path.join(dir_path, f"{name}.pai")
            cpai_file  = os.path.join(dir_path, f"{name}.cpai")
            npai_file  = os.path.join(dir_path, f"{name}.npai")

            data[name] = {}
            genome_sequence = next(SeqIO.parse(fasta_file, 'fasta'))
            data[name]['genome_sequence'] = genome_sequence

            with open(pai_file, 'r') as file:
                data[name]['pai_sequences'] = []

                while True:
                    line = file.readline()
                    if not line:
                        break

                    line = line.split()
                    pai_start_index = int(line[0])
                    pai_end_index = int(line[1])

                    pai_sequence = genome_sequence.seq[pai_start_index:pai_end_index]

                    data[name]['pai_sequences'].append((pai_sequence, (pai_start_index, pai_end_index)))

            with open(cpai_file, 'r') as file:
                data[name]['cpai_sequences'] = []

                while True:
                    line = file.readline()
                    if not line:
                        break

                    line = line.split()
                    cpai_start_index = int(line[0])
                    cpai_end_index = int(line[1])

                    cpai_sequence = genome_sequence.seq[cpai_start_index:cpai_end_index]

                    data[name]['cpai_sequences'].append((cpai_sequence, (cpai_start_index, cpai_end_index)))

            with open(npai_file, 'r') as file:
                data[name]['npai_sequences'] = []

                while True:
                    line = file.readline()
                    if not line:
                        break

                    line = line.split()
                    npai_start_index = int(line[0])
                    npai_end_index = int(line[1])

                    npai_sequence = genome_sequence.seq[npai_start_index:npai_end_index]

                    data[name]['npai_sequences'].append((npai_sequence, (npai_start_index, npai_end_index)))

    return data

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf(sequence, k):
    kmers = get_kmers(str(sequence), k)
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform([kmers])
    tfidf_matrix = X.toarray()
    return tfidf_matrix, vectorizer

def get_kmers(sequence, k):
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]
    return ' '.join(kmers)

def get_kmers_map(tfidf_matrix, vectorizer, sequence, top_n=20):
    feature_names = vectorizer.get_feature_names_out()
    feature_scores = tfidf_matrix.flatten()
    
    top_kmers_indices = feature_scores.argsort()[-top_n:][::-1]
    top_kmers = [(feature_names[i], feature_scores[i]) for i in top_kmers_indices]

    mapped_kmers = {}
    for kmer, score in top_kmers:
        kmer = kmer.upper()
        count = sequence.count(kmer)
        mapped_kmers[kmer] = {'score': score, 'count': count}
        
    return mapped_kmers

def find_unique_kmers(full_sequence, start_index, end_index, significant_kmers, unique_kmers):
    for kmer, details in significant_kmers.items():
        left_occurrences = full_sequence[:start_index].count(kmer)
        right_occurrences = full_sequence[end_index:].count(kmer)
        occurrences = left_occurrences + right_occurrences
        if occurrences == 0:
            unique_kmers.append((kmer, details['count']))

In [3]:
def get_unique_patterns(full_sequence, island_sequence, start_index, end_index, k_max):
    
    unique_kmers = []
    
    for k in range(4, k_max):
        tfidf_matrix, vectorizer = tfidf(island_sequence, k)
        kmers = get_kmers_map(tfidf_matrix, vectorizer, island_sequence)
        find_unique_kmers(full_sequence, start_index, end_index, kmers, unique_kmers)
        
    return unique_kmers

In [4]:
e_coli_data = read_data()

In [5]:
unique_pai_patterns  = {}
unique_cpai_patterns = {}
unique_npai_patterns = {}

k_max = 100
print(f'Max pattern length: {k_max}')
print('--------------------------')

for i, (name, data) in enumerate(e_coli_data.items()):
    print(f'Genome {i+1}/{len(e_coli_data)}: {name}')
    
    if len(data['pai_sequences']) != 0:
        unique_pai_patterns[name] = {}
        genome_sequence = data['genome_sequence']
        for j, (pai_sequence, indices) in enumerate(data['pai_sequences']):
            unique_patterns = get_unique_patterns(genome_sequence, pai_sequence, indices[0], indices[1], k_max)

            unique_pai_patterns[name]['pai_sequence'] = pai_sequence
            unique_pai_patterns[name]['indices'] = indices
            unique_pai_patterns[name]['patterns'] = unique_patterns

            print(f"PAI {j+1}/{len(data['pai_sequences'])}: {len(unique_patterns)} unique patterns")
    else:
        print('No PAI sequences')
    
    if len(data['cpai_sequences']) != 0:    
        unique_cpai_patterns[name] = {}
        genome_sequence = data['genome_sequence']
        for j, (cpai_sequence, indices) in enumerate(data['cpai_sequences']):
            unique_patterns = get_unique_patterns(genome_sequence, cpai_sequence, indices[0], indices[1], k_max)

            unique_cpai_patterns[name]['cpai_sequence'] = cpai_sequence
            unique_cpai_patterns[name]['indices'] = indices
            unique_cpai_patterns[name]['patterns'] = unique_patterns

            print(f"CPAI {j+1}/{len(data['cpai_sequences'])}: {len(unique_patterns)} unique patterns")
    else:
        print('No CPAI sequences')
        
    if len(data['npai_sequences']) != 0:
        unique_npai_patterns[name] = {}
        genome_sequence = data['genome_sequence']
        for j, (npai_sequence, indices) in enumerate(data['npai_sequences']):
            unique_patterns = get_unique_patterns(genome_sequence, npai_sequence, indices[0], indices[1], k_max)

            unique_npai_patterns[name]['npai_sequence'] = npai_sequence
            unique_npai_patterns[name]['indices'] = indices
            unique_npai_patterns[name]['patterns'] = unique_patterns

            print(f"NPAI {j+1}/{len(data['npai_sequences'])}: {len(unique_patterns)} unique patterns")
    else:
        print('No NPAI sequences')
    print('--------------------------')

Max pattern length: 100
--------------------------
Genome 1/90: NC_010468
No PAI sequences
CPAI 1/3: 1742 unique patterns
CPAI 2/3: 1735 unique patterns
CPAI 3/3: 631 unique patterns
NPAI 1/2: 1743 unique patterns
NPAI 2/2: 1746 unique patterns
--------------------------
Genome 2/90: NC_011748
No PAI sequences
CPAI 1/10: 641 unique patterns
CPAI 2/10: 1760 unique patterns
CPAI 3/10: 1751 unique patterns
CPAI 4/10: 1749 unique patterns
CPAI 5/10: 1619 unique patterns
CPAI 6/10: 581 unique patterns
CPAI 7/10: 36 unique patterns
CPAI 8/10: 1638 unique patterns
CPAI 9/10: 1648 unique patterns
CPAI 10/10: 1066 unique patterns
NPAI 1/1: 1740 unique patterns
--------------------------
Genome 3/90: NC_011751
PAI 1/1: 1454 unique patterns
CPAI 1/7: 657 unique patterns
CPAI 2/7: 1754 unique patterns
CPAI 3/7: 1738 unique patterns
CPAI 4/7: 1745 unique patterns
CPAI 5/7: 1753 unique patterns
CPAI 6/7: 1748 unique patterns
CPAI 7/7: 1399 unique patterns
NPAI 1/2: 1748 unique patterns
NPAI 2/2: 173

CPAI 3/8: 1752 unique patterns
CPAI 4/8: 1749 unique patterns
CPAI 5/8: 1744 unique patterns
CPAI 6/8: 1736 unique patterns
CPAI 7/8: 1743 unique patterns
CPAI 8/8: 1746 unique patterns
NPAI 1/3: 1749 unique patterns
NPAI 2/3: 1749 unique patterns
NPAI 3/3: 1750 unique patterns
--------------------------
Genome 34/90: NC_009800
No PAI sequences
CPAI 1/4: 654 unique patterns
CPAI 2/4: 1768 unique patterns
CPAI 3/4: 1407 unique patterns
CPAI 4/4: 1736 unique patterns
NPAI 1/1: 1688 unique patterns
--------------------------
Genome 35/90: NC_004431
PAI 1/2: 14 unique patterns
PAI 2/2: 4 unique patterns
CPAI 1/12: 664 unique patterns
CPAI 2/12: 1716 unique patterns


KeyboardInterrupt: 

In [ ]:
unique_pai_patterns